# DBSCAN spatial clustering on weather data by `Mr. Harshit Dawar`!
* It stands for Density Based Spatial clustering of applications with Noise.
* It works based on density of points in a particular radius.
* It is one of the most commonly used algorithms.
* It has 2 parameters on which it works:
  * Radius: It is the radius of the neirhborhood, if it includes enough points then it is known as Dense Area.
  * Minimum number of neighbors: These are the min. no. of points that should be present in the radius to define a cluster.

## Importing the libraries

In [2]:
import numpy as np 
import pandas as pd
from sklearn.cluster import DBSCAN 
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt 
import seaborn as sns

## Downlaoding the dataset!

In [3]:
!wget -O weather-stations20140101-20141231.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/weather-stations20140101-20141231.csv

--2020-05-31 01:46:27--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/weather-stations20140101-20141231.csv
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 129821 (127K) [text/csv]
Saving to: ‘weather-stations20140101-20141231.csv’

weather-stations201 100%[===================>] 126.78K   107KB/s    in 1.2s    

2020-05-31 01:46:29 (107 KB/s) - ‘weather-stations20140101-20141231.csv’ saved [129821/129821]



## Loading the dataset

In [4]:
data = pd.read_csv("weather-stations20140101-20141231.csv")